In [ ]:
!pip install pandas
import openai
import pandas as pd
from tqdm import tqdm
import os
from openai import OpenAI
tqdm.pandas(desc="Processing messages")

In [ ]:
client = OpenAI(api_key = api_key)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.3, 
    messages=[
        {"role": "system", "content": "You are a system for labeling messages coming from Telegram channels on military topics. Your task is to determine whether these messages contain military expertise. This information will be used by the Ukrainian military, it is important to catch all messages that contain information that directly or potentially indicates a change in the course of Russia's operation against Ukraine.  If the message mentions the work of Russian air defense, shelling of any Russian territories by Ukraine, internal events in Russia or Ukraine, statements by politicians or public figures on general topics, score 0 points. For each coincidence of the message with the established rule, add a point: 1 Description of Russia's battles and operations in Ukraine, their causes and consequences . 2 Mention the use, availability or description of any equipment. 3 Destruction of Ukrainian positions 4 Summarize the information in the form of reports or training materials. 5 Information on planning operations, coordination and organization of troops for combat missions and exercises. 6 Adaptation and improvement of tactics and methods in combat conditions based on experience. 7 Information on personnel and civilians, as well as resources that strengthen Russia's defense capabilities. 8 Things taken by the Russians as trophies 9 Recording successes, failures, or outcomes of events Example : красивое кадры работы су34 вкс с подвешенными фаб500м62 с умпк по позициям противника в зоне проведения сво публикует, войска движутся на юг.  Score Calculation: +1 for Rule 1 and +1 for Rule 2 and +1 for Rule 5. Your Output:3 Output only the number, without any additional comments."} , 
        {"role": "user", "content": "украину не будет смысла восстанавливать в случае ее поражения  так считает генсек нато что касается восстановления украины после войны то сначала нужно добиться чтобы она одержала верх если украина не одержит верх то будет нечего восстанавливать в свободной и независимой украине  заявил йенс столтенберг выступая на молодежном саммите нато  подписаться на izru"} , 
    ]
)


In [12]:
system_prompt = """You are a system for labeling messages coming from Telegram channels on military topics. 
Your task is to determine whether these messages contain military expertise. This information will be used 
by the Ukrainian military, it is important to catch all messages that contain information that directly or 
potentially indicates a change in the course of Russia's operation against Ukraine.  If the message mentions 
the work of Russian air defense, shelling of any Russian territories by Ukraine, internal events in Russia or 
Ukraine, statements by politicians or public figures on general topics, score 0 points. For each coincidence 
of the message with the established rule, add a point: 
1 Description of Russia's battles and operations in  Ukraine, their causes and consequences . 
2 Mention the use, availability or description of any equipment. 
3 Destruction of Ukrainian positions 
4 Summarize the information in the form of reports or training materials. 
5 Information on planning operations, coordination and organization of troops for combat missions and exercises. 
6 Adaptation and improvement of tactics and methods in combat conditions based on experience. 
7 Information on personnel and civilians, as well as resources that strengthen Russia's defense capabilities. 
8 Things taken by the Russians as trophies 
9 Recording successes, failures, or outcomes of events
Example : красивое кадры работы су34 вкс с подвешенными фаб500м62 с умпк по позициям противника в зоне проведения
 сво публикует, войска движутся на юг.  Score Calculation: +1 for Rule 1 and +1 for Rule 2 and +1 for Rule 5. 
 Your Output:3 Output only the number, without any additional comments."""

In [25]:
def get_score(message):
   # try:
        client = OpenAI(api_key = api_key)

        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0, 
            messages=[
                {"role": "system", "content": system_prompt} , 
                {"role": "user", "content": message} , 
            ]
        )

        score = response.choices[0].message.content

        if score.isdigit():
            return int(score)
        else:
            return None
    # except Exception as e:
    #     print(f"Error processing message: {e}")
    #     return None

In [26]:
get_score("украину не будет смысла восстанавливать в случае ее поражения  так считает генсек нато что касается восстановления украины после войны то сначала нужно добиться чтобы она одержала верх если украина не одержит верх то будет нечего восстанавливать в свободной и независимой украине  заявил йенс столтенберг выступая на молодежном саммите нато  подписаться на izru")

0

In [29]:
df = pd.read_csv("/Users/ulanagusar/Desktop/ML_week/df_sheets.csv")

In [33]:
def split_dataframe(df, chunk_size):
    chunks = []
    num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size != 0 else 0)
    for i in range(num_chunks):
        chunk = df.iloc[i*chunk_size : (i+1)*chunk_size]
        chunks.append(chunk)
    return chunks


chunk_size = 1000
chunks = split_dataframe(df, chunk_size)


output_dir = 'processed_chunks'
os.makedirs(output_dir, exist_ok=True)


for idx, chunk in enumerate(tqdm(chunks, desc="Processing chunks")):

    output_file = os.path.join(output_dir, f'dataset_with_scores_part{idx+1}.csv')
    if os.path.exists(output_file):
        print(f"Chunk {idx+1} already processed. Skipping...")
        continue


    tqdm.pandas(desc=f"Processing chunk {idx+1}")
    chunk['score'] = chunk['clean_message'].progress_apply(get_score)


    chunk.to_csv(output_file, index=False)
    print(f"Chunk {idx+1} saved to {output_file}")

print("Всі частини оброблено та збережено.")

Processing chunk 1: 100%|██████████| 1000/1000 [11:48<00:00,  1.41it/s]
/var/folders/nq/srzxpdh14hlgl_4b4d260fm40000gn/T/ipykernel_23090/2613046083.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['score'] = chunk['clean_message'].progress_apply(get_score)
Processing chunks:  12%|█▎        | 1/8 [11:48<1:22:42, 708.88s/it]

Chunk 1 saved to processed_chunks/dataset_with_scores_part1.csv


Processing chunk 2: 100%|██████████| 1000/1000 [12:31<00:00,  1.33it/s]
/var/folders/nq/srzxpdh14hlgl_4b4d260fm40000gn/T/ipykernel_23090/2613046083.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['score'] = chunk['clean_message'].progress_apply(get_score)
Processing chunks:  25%|██▌       | 2/8 [24:20<1:13:24, 734.14s/it]

Chunk 2 saved to processed_chunks/dataset_with_scores_part2.csv


Processing chunk 3: 100%|██████████| 1000/1000 [11:41<00:00,  1.43it/s]
/var/folders/nq/srzxpdh14hlgl_4b4d260fm40000gn/T/ipykernel_23090/2613046083.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['score'] = chunk['clean_message'].progress_apply(get_score)
Processing chunks:  38%|███▊      | 3/8 [36:02<59:56, 719.32s/it]  

...

Chunk 6 saved to processed_chunks/dataset_with_scores_part6.csv


Processing chunk 7: 100%|██████████| 1000/1000 [12:11<00:00,  1.37it/s]
/var/folders/nq/srzxpdh14hlgl_4b4d260fm40000gn/T/ipykernel_23090/2613046083.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['score'] = chunk['clean_message'].progress_apply(get_score)
Processing chunks:  88%|████████▊ | 7/8 [1:23:56<11:58, 718.73s/it]

Chunk 7 saved to processed_chunks/dataset_with_scores_part7.csv


Processing chunk 8: 100%|██████████| 484/484 [05:39<00:00,  1.42it/s]
/var/folders/nq/srzxpdh14hlgl_4b4d260fm40000gn/T/ipykernel_23090/2613046083.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['score'] = chunk['clean_message'].progress_apply(get_score)
Processing chunks: 100%|██████████| 8/8 [1:29:36<00:00, 672.05s/it]

Chunk 8 saved to processed_chunks/dataset_with_scores_part8.csv
Всі частини оброблено та збережено.
